In [2]:
import os
import cv2
import tasti
import torch
import pickle
import torchvision
import numpy as np
import pandas as pd
from tqdm.autonotebook import tqdm
from collections import defaultdict

os.environ['TORCH_HOME'] = '/lfs/1/jtguibas/models'
os.environ['FVCORE_CACHE'] = '/lfs/1/jtguibas/models'
        
config = tasti.IndexConfig()
config.do_mining = False
config.do_training = False
config.do_infer = False
config.do_bucketting = False

class VideoIndex(tasti.Index):
    def get_target_dnn(self):
        model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
        return model
        
    def get_embedding_dnn(self):
        model = torchvision.models.resnet18(pretrained=True, progress=True)
        model.fc = torch.nn.Linear(512, 128)
        return model
    
    def get_target_dnn_dataset(self):
        video = tasti.example.Video(
            video_fp='/lfs/1/jtguibas/data/2017-12-17',
            transform_fn=tasti.example.target_dnn_transform_fn # crop
        )
        return video
    
    def get_embedding_dnn_dataset(self):
        video = tasti.example.Video(
            video_fp='/lfs/1/jtguibas/data/2017-12-17',
            transform_fn=tasti.example.embedding_dnn_transform_fn # crop and resize
        )
        return video
    
    def override_target_dnn_cache(self, target_dnn_cache):
        labels = tasti.example.Labels(
            labels_fp='/lfs/1/jtguibas/data/labels/jackson-town-square-2017-12-17.csv',
            length=len(target_dnn_cache)
        )
        return labels
    
    def is_close(self, label1, label2):
        objects = set()
        for obj in (label1 + label2):
            objects.add(obj.object_name)
        for current_obj in list(objects):
            label1_disjoint = [obj for obj in label1 if obj.object_name == current_obj]
            label2_disjoint = [obj for obj in label2 if obj.object_name == current_obj]
            is_redundant = tasti.example.night_street_is_close_helper(label1_disjoint, label2_disjoint)
            if not is_redundant:
                return False
        return True
    
index = VideoIndex(config)
index.init()

In [3]:
query = tasti.AggregateQuery(index)
query.execute()

/lfs/1/jtguibas/tasti/blazeit/aggregation/samplers.py:62: RuntimeWarning: invalid value encountered in sqrt
  sigmat = np.sqrt(1/t * (Xt_sqsum - Xt_sum ** 2 / t))


Results
Initial Estimate: 325960.22655864124
Debiased Estimate: 350951.8614082856
Samples: 20454


{'initial_estimate': 325960.22655864124,
 'debiased_estimate': 350951.8614082856,
 'samples': 20454}

In [4]:
query = tasti.SUPGPrecisionQuery(index)
query.execute()


Results
Initial Estimate: 253610.02639515523
Debiased Estimate: 252542
idxs: [     0      1      2 ... 973076 973077 973079]
shape: (252542,)


{'initial_estimate': 253610.02639515523,
 'debiased_estimate': 252542,
 'idxs': array([     0,      1,      2, ..., 973076, 973077, 973079]),
 'shape': (252542,)}

In [5]:
query = tasti.SUPGRecallQuery(index)
query.execute()


Results
idxs: [     0      1      2 ... 973076 973077 973079]
shape: (250268,)


{'initial_estimate': 253610.02639515523,
 'debiased_estimate': 250268,
 'idxs': array([     0,      1,      2, ..., 973076, 973077, 973079]),
 'shape': (250268,)}